In [ ]:
#
# import utilitaires
#
import matplotlib.pyplot as plt
import pandas as pd
import random

import helpers as hlp
import helpers.dataset.PlantVillage as pv
import helpers.features as feat
import helpers.bovw as bovw

In [ ]:
#
# dataset PlantVillage
#
config_overrides = hlp.MetaObject.from_json("config_overrides.json")

pv_config = pv.Config()
if not config_overrides is None:
    hlp.MetaObject.override_from_object(pv_config, 
                                        config_overrides.dataset)

pv_dataset = pv.load(pv_config)
pv_dataframe = pv_dataset.dataframe

print("Dataset", pv_dataframe.shape)

#
# features extraction orb
#
orb_features_config = feat.OrbFeaturesConfig()
if not orb_features_config is None:
    hlp.MetaObject.override_from_object(orb_features_config, 
                                        config_overrides.orb.features)
orb_features_extraction = feat.ExtractionAdapter(orb_features_config)


#
# bag of visual words orb
#
orb_bovw_config = bovw.Config()
if not config_overrides is None:
    hlp.MetaObject.override_from_object(orb_bovw_config, 
                                        config_overrides.orb.bovw)

orb_bovw = bovw.load(orb_bovw_config, None)

print("ORB BoVW clusters:", orb_bovw.model.cluster_centers_.shape)

In [ ]:
#
# visualiser quelques orb keypoints
#
SAMPLES_GRID = (2, 4)
FIGSIZE = (10, SAMPLES_GRID[0] * 3.5)

# selectionner quelques images aleatoirement
random.seed(72)
image_indices = random.sample(range(pv_dataframe.shape[0]), SAMPLES_GRID[0] * SAMPLES_GRID[1])
image_indices.sort()


_, axes = plt.subplots(SAMPLES_GRID[0], SAMPLES_GRID[1], figsize=FIGSIZE)
for ax, image_index in zip(axes.flatten(), image_indices):
    plt.sca(ax)
    
    image = pv_dataset.get_image(image_index)    
    image_key_points, image_features = orb_features_extraction.extract(image)
    
    print( image_features.shape )
    
    species, disease = pv_dataframe.loc[image_index, ["species", "disease"]]
    disease = pv.attribute_prettify(species, disease)
    title = "\n".join([species, disease])
    
    plt.title(title)
    plt.axis("off")
    plt.imshow(image)

plt.suptitle("Image visual words")
plt.tight_layout()
plt.show()